<a href="https://colab.research.google.com/github/gtbnhyujmj/Gold_Cross-Exchange-Arbitrage/blob/main/XAUT_BX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import time
import requests
import hmac
from hashlib import sha256

In [43]:
# API 主網址
APIURL = "https://open-api.bingx.com"

In [44]:
# 你的 API 金鑰（可以先空著測試）
APIKEY = ""
SECRETKEY = ""

In [45]:
# 查詢 XAUT-USDT 的即時價格 (泰達公司出的泰達黃金)

def demo():
    # 請求資料可以是空的（GET 請求）
    payload = {}

    # API 的路徑
    path = '/openApi/swap/v1/ticker/price'

    # 請求方式
    method = "GET"

    # 要送出的參數
    paramsMap = {
        "symbol": "XAUT-USDT",  # 想查詢的幣種
        "timestamp": "1702718923479"  # 固定的測試值，後面會自動更新
    }

    # 將參數轉成 URL 格式字串
    paramsStr = parseParam(paramsMap)

    # 送出請求
    return send_request(method, path, paramsStr, payload)

In [46]:
# 對參數進行簽名（加密）

def get_sign(api_secret, payload):
    # 使用 SHA256 加密並回傳簽名值
    key_bytes = api_secret.encode("utf-8")
    payload_bytes = payload.encode("utf-8")
    signature = hmac.new(key_bytes, payload_bytes, digestmod=sha256).hexdigest()
    print("簽名結果 =", signature)
    return signature

In [47]:
# 發送 GET 請求

def send_request(method, path, urlpa, payload):
    # 手動組合 URL 字串（不使用 f-string）
    base_url = APIURL + path + "?" + urlpa + "&signature=" + get_sign(SECRETKEY, urlpa)
    print("請求網址 =", base_url)

    # 設定 headers，帶上 API 金鑰
    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    # 送出請求
    response = requests.request(method, base_url, headers=headers, data=payload)

    # 回傳伺服器的回應
    return response.text

In [48]:
# 將參數轉為 URL 字串，並自動加入目前時間戳

def parseParam(paramsMap):
    # 將參數依照 key 排序
    sortedKeys = sorted(paramsMap)

    # 將每組 key=value 用 & 串接起來
    paramsStr = ""
    for key in sortedKeys:
        if paramsStr != "":
            paramsStr = paramsStr + "&" + key + "=" + str(paramsMap[key])
        else:
            paramsStr = key + "=" + str(paramsMap[key])

    # 加上目前時間戳
    now_timestamp = "timestamp=" + str(int(time.time() * 1000))

    # 如果已經有參數，就補上 &timestamp
    if paramsStr != "":
        return paramsStr + "&" + now_timestamp
    else:
        return now_timestamp

In [49]:
# 主程式：呼叫 demo 函式
if __name__ == '__main__':
    print("執行結果:", demo())

簽名結果 = d8ccfa579737e7c0091120f6b4d83634c67f166a144c5fbc9e99e6f010dd2c5c
請求網址 = https://open-api.bingx.com/openApi/swap/v1/ticker/price?symbol=XAUT-USDT&timestamp=1702718923479&timestamp=1749907604041&signature=d8ccfa579737e7c0091120f6b4d83634c67f166a144c5fbc9e99e6f010dd2c5c
執行結果: {"code":0,"msg":"","data":{"symbol":"XAUT-USDT","price":"3433.7","time":1749907604208}}
